# Chapter 11 동적계획법


## 11.1 memoization

### 11.1.1 피보나치 수열

In [6]:
# benchmark fuction
from functools import wraps
import time

def benchmark(method):
    @wraps(method)
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print("{0}: {1:0.2f} ms".format(method.__name__, ((te-ts)*1000)))
        return result

    return timed

In [ ]:
from functools import wraps


def memo(func):
    cache = {}

    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

@memo
def fib2(n):
    if n < 2:
        return 1
    else:
        return fib2(n-1) + fib2(n-2)

def fib3(m, n):
    if m[n] == 0:
        m[n] = fib3(m, n-1) + fib3(m, n-2)
    return m[n]

@benchmark
def test_fib(n):
    print(fib(n))

@benchmark
def test_fib2(n):
    print(fib2(n))

@benchmark
def test_fib3(n):
    m = [0] * (n+1)
    m[0], m[1] = 1, 1
    print(fib3(m, n))

if __name__ == "__main__":
    n = 35
    test_fib(n)
    test_fib2(n)
    test_fib3(n)

14930352
test_fib: 3612.16 ms
14930352
test_fib2: 0.09 ms
14930352
test_fib3: 0.06 ms


decorator re

In [ ]:
from functools import wraps
#파이썬 데코레이터 예제
#wraps 사용 : 기존 정의한 함수 내 속성 유지
#wraps 미사용 : 기존 정의한 함수 내 속성 유지 하지 않음
#용도 : 특정 시점에 항상 실행되는 함수 정의 -> 로그, 통계성 데이터, 성능 테스트 등
 
def my_decorator1(func):
    def runs_func():
        print("decorator1 - no use wrapper")
        func()
    return runs_func
 
def my_decorator2(func):
    @wraps(func)
    def runs_func():
        print("decorator2 - use wrapper")
        func()
    return runs_func
 
@my_decorator1
def my_func1():
    '''
    func attr text1
    '''
    print("my_func1 run!")
 
@my_decorator2
def my_func2():
    '''
    func attr text2
    '''
    print("my_func2 run!")
 
 
#데코레이터 사용 함수 실행(wraps 미사용)
my_func1()
print("Func1 Name : ", my_func1.__name__)
print("Func1 Doc : ",  my_func1.__doc__)
 
#구분선
print('-' * 50)
 
#데코레이터 사용 함수 실행(wraps 사용)
my_func2()
print("Func2 Name : ", my_func2.__name__)
print("Func1 Doc : ",  my_func2.__doc__)
 
##### 참고 자료 #####
 
#데코레이터 미사용 함수 실행(wraps 미사용)
deco_my_func1 = my_decorator1(my_func1)
deco_my_func1()
 
#구분선
print('-' * 50)
 
#데코레이터 미사용 함수 실행(wraps 사용)
deco_my_func2 = my_decorator1(my_func2)
deco_my_func2()


decorator1 - no use wrapper
my_func1 run!
Func1 Name :  runs_func
Func1 Doc :  None
--------------------------------------------------
decorator2 - use wrapper
my_func2 run!
Func2 Name :  my_func2
Func1 Doc :  
    func attr text2
    
decorator1 - no use wrapper
decorator1 - no use wrapper
my_func1 run!
--------------------------------------------------
decorator1 - no use wrapper
decorator2 - use wrapper
my_func2 run!


데커레이더를 활용하면 데커레이터를 사용하는 함수의 __name__과 __doc__값에 차이가 생김.<br/> 파이썬에서 데커레이터를 사용한다면, 디버깅을 위해서 functools.wraps 모듈을 사용한다.

In [4]:
from functools import wraps

def logged(func):
    def with_logging(*args, **kwargs):
        """with_logging() 함수"""
        print(func.__name__ + " 호출")
        return func(*args, **kwargs)
    return with_logging

@logged
def f(x):
    """첫 번째, 데커레이터 사용 """
    return x + x * x

def f2(x):
    """두 번째, 데커레이터 사용 """
    return x + x * x

def logged2(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " 호출")
        return func(*args, **kwargs)
    return with_logging

@logged2
def f3(x):
    """세 번째, wraps와 데커레이터 사용 """
    return x + x * x

if __name__ == "__main__":
    print(f"결과: {f(5)}")
    print(f"__name__: {f.__name__}")
    print(f"__doc__: {f.__doc__}")
    print("_"*15)
    f2 = logged(f2)
    print(f"결과: {f2(5)}")
    print(f"__name__: {f2.__name__}")
    print(f"__doc__: {f2.__doc__}")
    print("-"*15)
    print(f"결과: {f3(5)}")
    print(f"__name__: {f3.__name__})")
    print(f"__doc__: {f3.__doc__}")


# 래핑해야만 다르게 뜸



f 호출
결과: 30
__name__: with_logging
__doc__: with_logging() 함수
_______________
f2 호출
결과: 30
__name__: with_logging
__doc__: with_logging() 함수
---------------
f3 호출
결과: 30
__name__: f3)
__doc__: 세 번째, wraps와 데커레이터 사용 


## 11.2 연습문제

### 11.2.1 최장 증가 부분열


In [14]:
from bisect import bisect
from itertools import combinations
from functools import wraps

# from benchmark import benchmark

def naive_longest_inc_subseq(seq):
    """ 1) 단순한 방법 """
    for length in range(len(seq), 0, -1):
        for sub in combinations(seq, length):        # 원하는 길이의 부분집합을 만들어줌
            if list(sub) == sorted(sub):
                return len(sub)

def dp_longest_inc_subseq(seq):      # 다시~
    """ 2) 동적 계획법 """
    L = [1] * len(seq)
    res = []

    for cur, val in enumerate(seq):
        for pre in range(cur):
            if seq[pre] <= val:
                L[cur] = max(L[cur], 1 + L[pre])
        return max(L)

def memo(func):
    cache = {}

    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

def memoized_longest_inc_subseq(seq):
    """ 3) 메모이제이션 """
    @memo
    def L(cur):
        res = 1
        for pre in range(cur):
            if seq[pre] <= seq[cur]:
                res = max(res, 1 + L(pre))
        return res
    return max(L(i) for i in range(len(seq)))

def longest_inc_bisec(seq):
    """ 4) 이진 검색 """
    end = []
    for val in seq:
        idx = bisect(end, val)
        if idx == len(end):
            end.append(val)
        else:
            end[idx] = val
        print(end)
    return len(end)

@benchmark
def test_naive_longest_inc_subseq():
    print(naive_longest_inc_subseq(s1))

@benchmark
def test_dp_longest_inc_subseq():
    print(dp_longest_inc_subseq(s1))

@benchmark
def test_memoized_longest_inc_subseq():
    print(memoized_longest_inc_subseq(s1))

@benchmark
def test_longest_inc_bisec():
    print(longest_inc_bisec(s1))

if __name__ == "__main__":
    # from random import randrange
    # s1 = [randrange(100) for i in range(20)]
    s1 = [94, 8, 78, 22,38, 79, 93, 8, 84, 39]
    print(s1)
    test_naive_longest_inc_subseq()
    test_dp_longest_inc_subseq()
    test_memoized_longest_inc_subseq()
    test_longest_inc_bisec()



            


[94, 8, 78, 22, 38, 79, 93, 8, 84, 39]
5
test_naive_longest_inc_subseq: 0.54 ms
1
test_dp_longest_inc_subseq: 0.25 ms
5
test_memoized_longest_inc_subseq: 0.23 ms
[94]
[8]
[8, 78]
[8, 22]
[8, 22, 38]
[8, 22, 38, 79]
[8, 22, 38, 79, 93]
[8, 8, 38, 79, 93]
[8, 8, 38, 79, 84]
[8, 8, 38, 39, 84]
5
test_longest_inc_bisec: 2.07 ms


In [11]:
s = [11, 15, 7, 35, 59, 43]
print(list(combinations(s, 3)))

s2 = enumerate(s)
for x, y in s2:
    print(x, y)

[(11, 15, 7), (11, 15, 35), (11, 15, 59), (11, 15, 43), (11, 7, 35), (11, 7, 59), (11, 7, 43), (11, 35, 59), (11, 35, 43), (11, 59, 43), (15, 7, 35), (15, 7, 59), (15, 7, 43), (15, 35, 59), (15, 35, 43), (15, 59, 43), (7, 35, 59), (7, 35, 43), (7, 59, 43), (35, 59, 43)]
0 11
1 15
2 7
3 35
4 59
5 43
